In [ ]:
import os
from tracktour import Tracker, get_im_centers

In [ ]:
DATA_ROOT = '/home/ddon0001/PhD/data/cell_tracking_challenge/ST/'
DS_NAME = 'Fluo-N2DL-HeLa/'
SEQ = '02_ST'
GT_PATH = os.path.join(DATA_ROOT, DS_NAME, '02_GT/TRA/')
SEG_PATH = os.path.join(DATA_ROOT, DS_NAME, SEQ, 'SEG/')

In [ ]:
seg_labels, detections, min_t, max_t, corners = get_im_centers(SEG_PATH)
print(f"Loaded {seg_labels.shape[0]} frames of shape {seg_labels.shape[1:]}.")
print(f"Extracted {len(detections)} detections:")
print(detections.head())

In [ ]:
tracker = Tracker(
    im_shape = corners[1],
    k_neighbours=10
)
tracked = tracker.solve(
    detections,
    frame_key="t",
    location_keys=("y", "x"),
    value_key="label"
)
print(tracked.tracked_detections.head())
print(tracked.tracked_edges.head())

In [ ]:
# convert solution to networkx graph
sol_graph = tracked.as_nx_digraph()

In [ ]:
# evaluate using `traccuracy` - note `traccuracy` should be installed separately!

from traccuracy import TrackingGraph
from traccuracy.loaders import load_ctc_data
from traccuracy.matchers import CTCMatcher
from traccuracy.metrics import CTCMetrics

comp_graph = TrackingGraph(
    sol_graph, 
    label_key="label", 
    location_keys=["y", "x"], 
    segmentation=seg_labels
)

gt_graph = load_ctc_data(GT_PATH)
matcher = CTCMatcher()
matched = matcher.compute_mapping(gt_graph, comp_graph)
results = CTCMetrics().compute(matched)

print(results.results)